In [5]:
%pip install ddgs langchain-experimental

Note: you may need to restart the kernel to use updated packages.


# Built-in-Tool

- DuckDuckGoSearch

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke('india vs pakistan match score')
results

'Watch Videos of India vs Pakistan , 6th Match , Group A in Dubai on September 14, 2025. India vs Pakistan – Asia Cup 2025: Match start time, team news and lineups. From head-to-head results to predicted XIs and tickets, here’s Al Jazeera’s guide to the India - Pakistan match in Dubai. India vs Pakistan Live Score : Spinners squeeze Salman Agha. The Indian spinners are squeezing the Pakistan batters here in the middle. Axar Patel is quite a canny bowler in this phase of the game, and with two right-handers in the middle, he will be tough to get away. The Indian captain produced a match -winning knock of 47 after the spinners helped India restrict Pakistan to just 127 in 20 overs. India will play Oman next Friday. India vs Pakistan , 6th Match , Group A. Dubai International Cricket Stadium, Dubai.'

In [17]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


- Shell Tool

In [8]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()
results = shell_tool.invoke('dir')
results

d:\anaconda3\envs\genai_env\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


Executing command:
 dir


' Volume in drive C has no label.\r\n Volume Serial Number is DE40-6C12\r\n\r\n Directory of c:\\Users\\pc\\Documents\\Coding\\Python\\Generative-AI\\langchain-tools\r\n\r\n09/15/2025  09:29 PM    <DIR>          .\r\n09/15/2025  09:29 PM    <DIR>          ..\r\n09/15/2025  09:36 PM            14,492 sample_tools.ipynb\r\n               1 File(s)         14,492 bytes\r\n               2 Dir(s)   8,900,714,496 bytes free\r\n'

# Custom Tools

### *Method-1: Using tool decorator*

In [13]:
from langchain_core.tools import tool

- **Step-1:** Create a function

In [9]:
def multiply(a, b):
    """Multiply two numbers"""
    return a*b

- **Step-2:** Add type hints

In [10]:
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

- **Step-3:** Add tool decorator

In [14]:
@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [15]:
result = multiply.invoke({'a':2, 'b':4})
result

8

In [16]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [ ]:
multiply.args_schema.model_json_schema()        # this goes to LLM

{'description': 'Multiply two numbers',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'multiply',
 'type': 'object'}

### *Method-2: Using StructureTool*

In [19]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [20]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [21]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [22]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [23]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


### *Method-3: Using BaseTool*

In [24]:
from langchain.tools import BaseTool
from typing import Type

In [25]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [26]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [27]:
multiply_tool = MultiplyTool()

In [28]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## Toolkit

In [29]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [30]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]

In [31]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)

add => Add two numbers
multiply => Multiply two numbers
